In [1]:
import asyncio
import os
from pydantic import BaseModel
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential

In [2]:
class OutputStruct(BaseModel):
    """A structured output for testing purposes."""

    city: str
    description: str

In [3]:
print("=== Non-streaming example ===")

credential = AzureCliCredential()
client = AzureOpenAIResponsesClient(
    project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    deployment_name=os.environ["AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME"],
    credential=credential,
)


agent = client.as_agent(
    name="CityAgent",
    instructions="You are a helpful agent that describes cities in a structured format.",
)

=== Non-streaming example ===


In [4]:
query = "Tell me about Paris, France"
print(f"User: {query}")

User: Tell me about Paris, France


In [5]:
result = await agent.run(query, options={"response_format": OutputStruct})

if structured_data := result.value:
    print("Structured Output Agent:")
    print(f"City: {structured_data.city}")
    print(f"Description: {structured_data.description}")
else:
    print(f"Failed to parse response: {result.text}")

Structured Output Agent:
City: Paris
Description: Paris, the capital city of France, is renowned for its art, fashion, gastronomy, and culture. It's home to iconic landmarks like the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. Known as the "City of Light," Paris is a global hub for art and design, with charming streets, cafes, and a rich history evident in its architecture and museums. It's also famous for its culinary scene, with a plethora of cafes and restaurants offering exquisite French cuisine.


In [6]:
print("=== Streaming example ===")

agent = client.as_agent(
    name="CityAgent",
    instructions="You are a helpful agent that describes cities in a structured format.",
)

query = "Tell me about Tokyo, Japan"
print(f"User: {query}")

=== Streaming example ===
User: Tell me about Tokyo, Japan


In [7]:
stream = agent.run(query, stream=True, options={"response_format": OutputStruct})
async for update in stream:
    if update.text:
        print(update.text, end="", flush=True)
print()

{"city":"Tokyo","description":"Tokyo is the capital city of Japan, known for its blend of traditional culture and modern innovation. It features landmarks like the historic Imperial Palace, the bustling Shibuya Crossing, and the iconic Tokyo Tower. The city offers a diverse range of experiences, from high-tech shopping in Akihabara to serene walks in the gardens of Shinjuku Gyoen. Tokyo is also renowned for its culinary scene, offering everything from sushi to ramen to international cuisine. As a global financial center, it hosts numerous multinational companies and is a hub for technology and finance."}


In [8]:
result = await stream.get_final_response()

if structured_data := result.value:
    print("Structured Output (from streaming with ResponseStream):")
    print(f"City: {structured_data.city}")
    print(f"Description: {structured_data.description}")
else:
    print(f"Failed to parse response: {result.text}")

Structured Output (from streaming with ResponseStream):
City: Tokyo
Description: Tokyo is the capital city of Japan, known for its blend of traditional culture and modern innovation. It features landmarks like the historic Imperial Palace, the bustling Shibuya Crossing, and the iconic Tokyo Tower. The city offers a diverse range of experiences, from high-tech shopping in Akihabara to serene walks in the gardens of Shinjuku Gyoen. Tokyo is also renowned for its culinary scene, offering everything from sushi to ramen to international cuisine. As a global financial center, it hosts numerous multinational companies and is a hub for technology and finance.
